In [90]:
%load_ext autoreload
%autoreload 2

import os
import pymc as pm
import arviz as az
import numpy as np
import pandas as pd
import pytensor
from pytensor import tensor as T
from sklearn.preprocessing import scale, StandardScaler, LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.cluster.hierarchy import linkage, leaves_list
import itertools
import pickle
import dill
from pyprojroot.here import here
import numpyro

numpyro.enable_x64()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
here('submission/draft/survival_clustering.ipynb')

PosixPath('/Users/alzhang/Documents/projects/tfri_halo/submission/draft/survival_clustering.ipynb')

In [3]:
def create_trace_table(trace, export_variables = ['props', 'beta_clust', 'beta_stage', 'beta_age', 'beta_chemo', 'beta_rt', 'beta_brachy', 'beta_histotype']):
    # Create an empty list to store DataFrames
    export_dfs = []
    
    # Iterate over the list of column names
    for variable in export_variables:
        print(variable)
        posterior = trace.posterior[variable][0]
        
        if posterior.ndim == 2:
            posterior_clust_assignments = pd.DataFrame(posterior, columns=[f'{variable}_{i}' for i in range(posterior.shape[1])])
        elif posterior.ndim == 1:
            posterior_clust_assignments = pd.DataFrame(posterior, columns=[variable])
        else:
            print("Should not get here.")
        
        # Append the DataFrame to the list
        export_dfs.append(posterior_clust_assignments)
    
    # Concatenate the DataFrames horizontally (column bind)
    trace_table = pd.concat(export_dfs, axis=1)
    return trace_table

## Inputs

In [4]:
# Survival data, long format by outcome
clinical_long = pd.read_csv(here('results/survival_cluster/clinical_long.tsv'), sep='\t')

# Counts data, long format by TIL type and region
counts_long = pd.read_csv(here('results/survival_cluster/counts_final.tsv'), sep='\t')

In [5]:
# Filter for p53abn
clinical_long = clinical_long.loc[clinical_long['eclass2_ngs'] == 'p53abn'].dropna()

eclass_encoder = LabelEncoder()
stage_encoder = LabelEncoder()

clinical_long['eclass2_ngs_idx'] = eclass_encoder.fit_transform(clinical_long['eclass2_ngs'])
clinical_long['stage_idx'] = stage_encoder.fit_transform(clinical_long['stage_main'])
clinical_long['carcinosarcoma'] = (clinical_long['hist_rev'] == 'carcinosarcoma (MMMT)')
clinical_long['age_dx'] = scale(clinical_long['age_dx'], axis=0, with_mean=True, with_std=True, copy=True)

clinical_vars = ["acc_num", "outcome", "time", "status", "chemo", "rt", "brachy", "eclass2_ngs_idx", "stage_idx", "age_dx", "carcinosarcoma"]

clinical_selected = clinical_long[clinical_vars].drop_duplicates()

In [6]:
counts_long['variable_region'] = counts_long['variable'] + '_' + counts_long['region']

counts_wide = counts_long.pivot(index='acc_num', columns='variable_region', values='value').dropna()
areas_wide = counts_long.pivot(index='acc_num', columns='variable_region', values='area_region_mm').dropna()

In [7]:
# Accepts counts and areas in wide format, clinical data in long format
# Returns clinical data, areas, and counts indexed identically
def get_inputs(counts, areas, clinical, outcome):
    clinical = clinical[clinical['outcome'] == outcome].set_index('acc_num')
    
    common_samples = np.intersect1d(counts.index, clinical.index)
    counts = counts.loc[common_samples,:]
    areas = areas.loc[common_samples,:]
    clinical = clinical.loc[common_samples,:]

    return {'clinical': clinical, 'areas': areas, 'counts': counts}

In [8]:
# Output N x C (number of samples X number of region*cell types) counts array, with corresponding N x C area array
# Note that this is DIFFERENT from previous

os_inputs = get_inputs(counts_wide, areas_wide, clinical_selected, outcome = 'os')
pfs_inputs = get_inputs(counts_wide, areas_wide, clinical_selected, outcome = 'pfs')
dss_inputs = get_inputs(counts_wide, areas_wide, clinical_selected, outcome = 'dss')

In [44]:
def fit_survcluster_model(inputs, nclusts = 2, ncenters = 20, interval_length = 0.3, epsilon = 1e-6, ndraw = 1000, ntune=1000):
    count_mat = np.array(inputs['counts'])
    area_mat = np.array(inputs['areas'])
    clinical_df = inputs['clinical']
    time = clinical_df['time'].values
    event = clinical_df['status'].values
    stage = clinical_df['stage_idx'].values
    age = clinical_df['age_dx'].values
    chemo = clinical_df['chemo'].values.astype(int)
    rt = clinical_df['rt'].values.astype(int)
    brachy = clinical_df['brachy'].values.astype(int)
    
    nstages = len(np.unique(stage))
    ncelltypes = count_mat.shape[1]
    nsamples = count_mat.shape[0]
    
    mean_mu = np.sum(count_mat, axis = 0)/np.sum(area_mat, axis = 0)
    mean_mu = np.repeat(mean_mu[np.newaxis,:], nclusts, axis=0).transpose(0, 1)
    
    area_nonzero = area_mat + epsilon
    
    rbf_step = np.max(count_mat)/(ncenters-1.)
    centers = np.arange(ncenters) * rbf_step
    
    # intervals 
    
    samples = np.arange(nsamples)
    interval_bounds = np.arange(0, time.max() + interval_length + 1, interval_length)
    
    nintervals = interval_bounds.size - 1
    intervals = np.arange(nintervals)
    
    last_period = np.floor((time - 0.01) / interval_length).astype(int)
    
    death = np.zeros((nsamples, nintervals))
    death[samples, last_period] = event
    
    exposure = np.greater_equal.outer(time, interval_bounds[:-1]) * interval_length
    exposure[samples, last_period] = time - interval_bounds[last_period]
    
    coords = {"intervals": intervals}

    with pm.Model(coords = coords) as survival_mixture_model:
        # Priors for survival coefficients
        
        beta_clust0 = pm.Normal("beta_clust0", mu=0, sigma=5, shape=nclusts-1)
        beta_clust = pm.Deterministic("beta_clust", pm.math.concatenate([[0], beta_clust0]))
        beta_stage0 = pm.Normal("beta_stage0", mu=0, sigma=5, shape=nstages-1)
        beta_stage = pm.Deterministic("beta_stage", pm.math.concatenate([[0], beta_stage0]))
        beta_age = pm.Normal("beta_age", mu=0, sigma=5)
        beta_chemo = pm.Normal("beta_chemo", mu = 0, sigma = 5)
        beta_rt = pm.Normal("beta_rt", mu = 0, sigma = 5)
        beta_brachy = pm.Normal("beta_brachy", mu = 0, sigma = 5)
    
        # Priors for count coefficient
        mu_clust = pm.Gamma("mu_clust", mu = mean_mu, sigma = 100, shape = (nclusts, ncelltypes))
        
        # Latent categorical variable for 'clust'
        props = pm.Dirichlet('props', np.ones(nclusts))
        clust = pm.Categorical("clust", p=props, shape=nsamples)
    
        # NB distribution for count data using adjusted area and mu_clust
        count_mu = area_nonzero * mu_clust[clust]  # Use 'area' as multiplier
    
        # RBF based NB dispersion 
        theta_a = pm.Normal("theta_a", mu = 0, sigma=1, shape=ncenters)
        theta_b = pm.Normal("theta_b", mu = 0, sigma=1, shape=ncenters)
        count_disp = T.dot(T.exp(-T.exp(theta_b) * (T.reshape(T.repeat(count_mu, repeats=ncenters), newshape=(nsamples, ncelltypes, ncenters)) - centers[np.newaxis, np.newaxis, :])**2), T.exp(theta_a)) + epsilon
    
        # Likelihood for count data
        count_v = pm.NegativeBinomial("count_v", mu=count_mu, alpha=count_disp, observed=count_mat)
        
        # Don't forget lambda_0 AFTERWARDS ADD THIS -- need to fix intercepts and dimensions as a result
        lambda0 = pm.Gamma("lambda0", 0.1, 0.1, dims = "intervals") # 0.01, 0.05, 0.1
        
        # Linear predictor for Cox PH model
        linear_predictor = (
            beta_clust[clust]
            + beta_stage[stage]
            + beta_age * age
            + beta_chemo * chemo
            + beta_rt * rt
            + beta_brachy * brachy
        )
    
    
        lambda_ = pm.Deterministic("lambda_", T.outer(T.exp(linear_predictor), lambda0))
    
        mu = pm.Deterministic("mu", exposure * lambda_)
    
        # Poisson-Cox PH equivalence
        obs = pm.Poisson("obs", mu=mu, observed=death)
    
        survival_mixture_model.debug(verbose = True)
    
        trace = pm.sample(ndraw, tune=ntune, chains=1, progressbar=True)

        return {'trace': trace, 'model': survival_mixture_model}

In [55]:
os_outputs = fit_survcluster_model(os_inputs, nclusts = 2, ncenters = 20, interval_length = 0.3, epsilon = 1e-6, ndraw = 1000, ntune=1000)

point={'beta_clust0': array([0.]), 'beta_stage0': array([0., 0., 0.]), 'beta_age': array(0.), 'beta_chemo': array(0.), 'beta_rt': array(0.), 'beta_brachy': array(0.), 'mu_clust_log__': array([[5.74134442, 2.3838835 , 4.46843508, 1.23147393, 2.59370369,
        1.00939819, 5.61453408, 4.55177006, 2.46390665, 1.13416853,
        5.53723606, 3.99462307, 3.63119581, 2.00995528, 4.84529417,
        2.38146401, 5.03593986, 2.95538798, 4.79499169, 3.24137003,
        6.13982856, 4.427772  , 6.11182582, 3.85459295, 5.21345766,
        2.7528743 ],
       [5.74134442, 2.3838835 , 4.46843508, 1.23147393, 2.59370369,
        1.00939819, 5.61453408, 4.55177006, 2.46390665, 1.13416853,
        5.53723606, 3.99462307, 3.63119581, 2.00995528, 4.84529417,
        2.38146401, 5.03593986, 2.95538798, 4.79499169, 3.24137003,
        6.13982856, 4.427772  , 6.11182582, 3.85459295, 5.21345766,
        2.7528743 ]]), 'props_simplex__': array([0.]), 'clust': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

Sequential sampling (1 chains in 1 job)
CompoundStep
>NUTS: [beta_clust0, beta_stage0, beta_age, beta_chemo, beta_rt, beta_brachy, mu_clust, props, theta_a, theta_b, lambda0]
>BinaryGibbsMetropolis: [clust]


Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 622 seconds.
Only one chain was sampled, this makes it impossible to run some convergence checks


In [56]:
pm.summary(os_outputs['trace'], var_names=["props", "beta_clust", "beta_stage", "beta_age", "beta_chemo", "beta_rt", "beta_brachy"])

arviz - WARNING - Shape validation failed: input_shape: (1, 1000), minimum_shape: (chains=2, draws=4)


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
props[0],0.509,0.038,0.439,0.577,0.004,0.003,91.0,541.0,NaN
props[1],0.491,0.038,0.423,0.561,0.004,0.003,91.0,541.0,NaN
beta_clust[0],0.000,0.000,0.000,0.000,0.000,0.000,1000.0,1000.0,NaN
beta_clust[1],-0.447,0.253,-0.945,0.017,0.008,0.006,956.0,733.0,NaN
beta_stage[0],0.000,0.000,0.000,0.000,0.000,0.000,1000.0,1000.0,NaN
beta_stage[1],0.625,0.694,-0.609,1.938,0.024,0.018,956.0,613.0,NaN
beta_stage[2],1.063,0.333,0.397,1.642,0.014,0.011,568.0,581.0,NaN
beta_stage[3],2.894,0.440,2.069,3.745,0.023,0.017,360.0,446.0,NaN
beta_age,0.205,0.127,-0.017,0.454,0.004,0.003,1042.0,588.0,NaN
beta_chemo,-0.872,0.330,-1.458,-0.250,0.013,0.009,641.0,566.0,NaN


In [51]:
pfs_outputs = fit_survcluster_model(pfs_inputs, nclusts = 2, ncenters = 20, interval_length = 0.3, epsilon = 1e-6, ndraw = 1000, ntune=1000)

point={'beta_clust0': array([0.]), 'beta_stage0': array([0., 0., 0.]), 'beta_age': array(0.), 'beta_chemo': array(0.), 'beta_rt': array(0.), 'beta_brachy': array(0.), 'mu_clust_log__': array([[5.68196172, 2.24199047, 4.48661082, 1.18186252, 2.61167384,
        1.02248025, 5.59179312, 4.53887048, 2.45747758, 1.10592862,
        5.5459417 , 4.01413411, 3.62124397, 2.01490522, 4.83820893,
        2.37418687, 5.05412827, 2.97242243, 4.77933392, 3.23039384,
        6.15899309, 4.37556219, 6.12371957, 3.82459632, 5.22114322,
        2.75491914],
       [5.68196172, 2.24199047, 4.48661082, 1.18186252, 2.61167384,
        1.02248025, 5.59179312, 4.53887048, 2.45747758, 1.10592862,
        5.5459417 , 4.01413411, 3.62124397, 2.01490522, 4.83820893,
        2.37418687, 5.05412827, 2.97242243, 4.77933392, 3.23039384,
        6.15899309, 4.37556219, 6.12371957, 3.82459632, 5.22114322,
        2.75491914]]), 'props_simplex__': array([0.]), 'clust': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

Sequential sampling (1 chains in 1 job)
CompoundStep
>NUTS: [beta_clust0, beta_stage0, beta_age, beta_chemo, beta_rt, beta_brachy, mu_clust, props, theta_a, theta_b, lambda0]
>BinaryGibbsMetropolis: [clust]


Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 584 seconds.
Only one chain was sampled, this makes it impossible to run some convergence checks


In [53]:
pm.summary(pfs_outputs['trace'], var_names=["props", "beta_clust", "beta_stage", "beta_age", "beta_chemo", "beta_rt", "beta_brachy"])

arviz - WARNING - Shape validation failed: input_shape: (1, 1000), minimum_shape: (chains=2, draws=4)


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
props[0],0.494,0.036,0.427,0.562,0.001,0.001,787.0,739.0,NaN
props[1],0.506,0.036,0.438,0.573,0.001,0.001,787.0,739.0,NaN
beta_clust[0],0.000,0.000,0.000,0.000,0.000,0.000,1000.0,1000.0,NaN
beta_clust[1],-0.430,0.235,-0.887,0.004,0.008,0.006,877.0,771.0,NaN
beta_stage[0],0.000,0.000,0.000,0.000,0.000,0.000,1000.0,1000.0,NaN
beta_stage[1],-0.215,0.833,-1.823,1.182,0.027,0.028,1111.0,444.0,NaN
beta_stage[2],1.509,0.314,0.944,2.091,0.012,0.009,749.0,601.0,NaN
beta_stage[3],2.865,0.430,2.071,3.660,0.018,0.013,569.0,474.0,NaN
beta_age,0.013,0.124,-0.236,0.219,0.004,0.004,1115.0,680.0,NaN
beta_chemo,-0.470,0.346,-1.083,0.196,0.014,0.010,649.0,578.0,NaN


In [59]:
dss_outputs = fit_survcluster_model(dss_inputs, nclusts = 2, ncenters = 20, interval_length = 0.3, epsilon = 1e-6, ndraw = 1000, ntune=1000)

point={'beta_clust0': array([0.]), 'beta_stage0': array([0., 0., 0.]), 'beta_age': array(0.), 'beta_chemo': array(0.), 'beta_rt': array(0.), 'beta_brachy': array(0.), 'mu_clust_log__': array([[5.75870179, 2.31838358, 4.46375287, 1.24355049, 2.60599007,
        1.03637965, 5.60291106, 4.55877398, 2.46794208, 1.15072592,
        5.54594424, 4.0232128 , 3.6484534 , 2.04259638, 4.85501928,
        2.39959025, 5.05330594, 2.98003355, 4.80937621, 3.27254467,
        6.14341007, 4.42421188, 6.11291037, 3.83479628, 5.2219894 ,
        2.75504639],
       [5.75870179, 2.31838358, 4.46375287, 1.24355049, 2.60599007,
        1.03637965, 5.60291106, 4.55877398, 2.46794208, 1.15072592,
        5.54594424, 4.0232128 , 3.6484534 , 2.04259638, 4.85501928,
        2.39959025, 5.05330594, 2.98003355, 4.80937621, 3.27254467,
        6.14341007, 4.42421188, 6.11291037, 3.83479628, 5.2219894 ,
        2.75504639]]), 'props_simplex__': array([0.]), 'clust': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

Sequential sampling (1 chains in 1 job)
CompoundStep
>NUTS: [beta_clust0, beta_stage0, beta_age, beta_chemo, beta_rt, beta_brachy, mu_clust, props, theta_a, theta_b, lambda0]
>BinaryGibbsMetropolis: [clust]


Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 755 seconds.
Only one chain was sampled, this makes it impossible to run some convergence checks


In [60]:
pm.summary(dss_outputs['trace'], var_names=["props", "beta_clust", "beta_stage", "beta_age", "beta_chemo", "beta_rt", "beta_brachy"])

arviz - WARNING - Shape validation failed: input_shape: (1, 1000), minimum_shape: (chains=2, draws=4)


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
props[0],0.513,0.038,0.440,0.580,0.002,0.001,551.0,559.0,NaN
props[1],0.487,0.038,0.420,0.560,0.002,0.001,551.0,559.0,NaN
beta_clust[0],0.000,0.000,0.000,0.000,0.000,0.000,1000.0,1000.0,NaN
beta_clust[1],-0.481,0.287,-0.970,0.124,0.010,0.008,865.0,628.0,NaN
beta_stage[0],0.000,0.000,0.000,0.000,0.000,0.000,1000.0,1000.0,NaN
beta_stage[1],0.099,1.364,-2.207,2.542,0.052,0.051,806.0,476.0,NaN
beta_stage[2],1.618,0.399,0.865,2.297,0.016,0.012,618.0,577.0,NaN
beta_stage[3],3.449,0.540,2.494,4.444,0.024,0.017,521.0,641.0,NaN
beta_age,0.093,0.140,-0.168,0.365,0.005,0.004,849.0,618.0,NaN
beta_chemo,-0.991,0.402,-1.814,-0.305,0.015,0.011,739.0,569.0,NaN


## Output traces

In [81]:
def create_trace_table(trace, export_variables = ['props', 'beta_clust', 'beta_stage', 'beta_age', 'beta_chemo', 'beta_rt', 'beta_brachy']):
    # Create an empty list to store DataFrames
    export_dfs = []
    
    # Iterate over the list of column names
    for variable in export_variables:
        #print(variable)
        posterior = trace.posterior[variable][0]
        
        if posterior.ndim == 2:
            posterior_clust_assignments = pd.DataFrame(posterior, columns=[f'{variable}_{i}' for i in range(posterior.shape[1])])
        elif posterior.ndim == 1:
            posterior_clust_assignments = pd.DataFrame(posterior, columns=[variable])
        else:
            print("Should not get here.")
        
        # Append the DataFrame to the list
        export_dfs.append(posterior_clust_assignments)
    
    # Concatenate the DataFrames horizontally (column bind)
    trace_table = pd.concat(export_dfs, axis=1)
    return trace_table

def extract_cluster_assignments(trace, sample_names):
    cluster_df = pd.DataFrame(trace.posterior['clust'][0].T)
    cluster_df.index = sample_names
    return cluster_df.reset_index()

In [83]:
os_trace_table = create_trace_table(os_outputs['trace'])
os_cluster_assignments = extract_cluster_assignments(os_outputs['trace'], sample_names = os_inputs['counts'].index)

pfs_trace_table = create_trace_table(pfs_outputs['trace'])
pfs_cluster_assignments = extract_cluster_assignments(pfs_outputs['trace'], sample_names = pfs_inputs['counts'].index)

dss_trace_table = create_trace_table(dss_outputs['trace'])
dss_cluster_assignments = extract_cluster_assignments(dss_outputs['trace'], sample_names = dss_inputs['counts'].index)

In [91]:
trace_output_dir = here('results/survival_cluster/traces')
#model_output_dir = here('results/survival_cluster/models')

os_trace_table.to_csv(os.path.join(trace_output_dir, 'os_trace_table.tsv'), sep='\t')
os_cluster_assignments.to_csv(os.path.join(trace_output_dir, 'os_cluster_assignments.tsv'), sep='\t')

pfs_trace_table.to_csv(os.path.join(trace_output_dir, 'pfs_trace_table.tsv'), sep='\t')
pfs_cluster_assignments.to_csv(os.path.join(trace_output_dir, 'pfs_cluster_assignments.tsv'), sep='\t')

dss_trace_table.to_csv(os.path.join(trace_output_dir, 'dss_trace_table.tsv'), sep='\t')
dss_cluster_assignments.to_csv(os.path.join(trace_output_dir, 'dss_cluster_assignments.tsv'), sep='\t')

# Output arviz inferencedata objects
os_outputs['trace'].to_netcdf(os.path.join(trace_output_dir, 'os_results.nc'))
pfs_outputs['trace'].to_netcdf(os.path.join(trace_output_dir, 'pfs_results.nc'))
dss_outputs['trace'].to_netcdf(os.path.join(trace_output_dir, 'dss_results.nc'))

'/Users/alzhang/Documents/projects/tfri_halo/results/survival_cluster/traces/dss_results.nc'